In [28]:
import requests
import configparser
import pandas as pd
import requests

In [17]:
def get_apikey():
    
    config = configparser.ConfigParser()
    config.read('../config.ini')
    
    return config['DEFAULT']['APIKEY']

In [23]:
def get_symbol_list():
    
    df = pd.read_csv('../data/static/sandp100.csv')
    
    return df['Ticker'].tolist()

In [29]:
def api_call(function="TIME_SERIES_DAILY"):
    
    api_key = get_apikey()
    symbol_list = get_symbol_list()[:1]
    
    for symbol in symbol_list:
        url = f"https://www.alphavantage.co/query?function={function}&symbol={symbol}&apikey={api_key}"
        response = requests.get(url)
        print(response.json())

In [30]:
api_call()

{'Meta Data': {'1. Information': 'Daily Prices (open, high, low, close) and Volumes', '2. Symbol': 'MMM', '3. Last Refreshed': '2021-09-22', '4. Output Size': 'Compact', '5. Time Zone': 'US/Eastern'}, 'Time Series (Daily)': {'2021-09-22': {'1. open': '179.3100', '2. high': '180.9479', '3. low': '178.9400', '4. close': '179.8400', '5. volume': '2220767'}, '2021-09-21': {'1. open': '180.9700', '2. high': '181.0800', '3. low': '177.7000', '4. close': '178.4200', '5. volume': '2980077'}, '2021-09-20': {'1. open': '178.6400', '2. high': '181.7400', '3. low': '178.2900', '4. close': '180.5300', '5. volume': '4152261'}, '2021-09-17': {'1. open': '181.0900', '2. high': '182.6599', '3. low': '179.4100', '4. close': '181.4900', '5. volume': '10719870'}, '2021-09-16': {'1. open': '184.1500', '2. high': '184.6000', '3. low': '181.8243', '4. close': '182.6600', '5. volume': '2553628'}, '2021-09-15': {'1. open': '182.8200', '2. high': '185.1100', '3. low': '182.3300', '4. close': '184.1900', '5. vol